### https://www.thedetailingmafia.com/locate-our-outlets

In [157]:
# importing required packages
import pandas as pd
import requests
import concurrent.futures
import numpy as np
import sys
from os.path import exists
import re
import itertools
import orjson
import time
from datetime import date
import urllib.parse
import os
import xml.etree.ElementTree
from bs4 import BeautifulSoup
from os.path import exists
import json
import csv
import time

In [158]:
CLEANR = re.compile('<.*?>')
def cleanhtml(raw_html):
    cleantext =re.sub(CLEANR, '', str(raw_html))
    return cleantext

In [159]:
out_csv = 'DetailingMafia.csv'
input_folder = "DetailingMafia"
baseurl = "https://www.thedetailingmafia.com/locate-our-outlets"

csv_exists = exists(out_csv)
if csv_exists:
    print(f"csv exists in path {out_csv},removing now ....")
    os.remove(out_csv)
    
df = pd.DataFrame(columns=['Company_Name','Pincode','State','City','Company_Url','Logo_Url','Rating,Reviews','Contact_Person_1','Contact_Person_2','Email','Website','Address','Head_Office','Listed_In,About_Us','Opening_Closing_Time','Weekly_Services','Mobile1','Mobile2','Mobile3','Mobile4','Mobile5','Mobile6','tollfree1','tollfree2','tollfree3','stdcode','Phone1','Phone2','Phone3','Phone4','Phone5','fax_cut','Fax_prefix','Fax_Number1','Fax_Number2','Tag,Image_Urls','Established_Year','Tagline','Feature-Tags','Tagline2','Lat','Lon'])
header_list=df.columns
print(f"creating new CSV...")
df.to_csv(out_csv,sep="|",index=False,header=False)

csv exists in path DetailingMafia.csv,removing now ....
creating new CSV...


In [160]:
parent_dir = "/home/justdial/data/input"
path = os.path.join(parent_dir, input_folder) 
file_exists = exists(path)
if not file_exists :
        os.makedirs(path)
else:
    print("Path Already Exits")

Path Already Exits


### fetch_Details() function

In [161]:
def fetch_details(spage,main_cat):
    spage = re.sub('<!--(.*?)-->','',str(spage))
    spage = re.sub('&times;','x',str(spage))
    spage = re.sub('amp;','',str(spage))
    spage = re.sub('&#8211;','-',str(spage))
    spage = re.sub('&#8217;','',str(spage))
    spage = re.sub('&#038;','&',str(spage))
    spage = re.sub('&#8243;','Inches',str(spage))
    #spage = re.sub('&quot;','Inches',str(spage))
    spage = re.sub('&nbsp;','',str(spage))
    
    # cut
    cut = re.findall('<div class="mapcontainer">(.*?)<div class="row outletAddress">',str(spage))
    
    # company name
    try:
        company_name = re.findall('<h2>(.*?)</h2>',str(cut))[0]
        company_name = re.sub('</li>\s*</ul>','',str(company_name))
        company_name = re.sub('</li>',' ~ ',str(company_name))
        #company_name = " ".join(company_name)
        company_name = cleanhtml(company_name)
    
    except:
        company_name = ""
    
    # city
    try:
        city = re.findall('<h2><span>(.*?)</span></h2>',str(cut))[0]
        city = re.sub('</li>\s*</ul>','',str(city))
        city = re.sub('</li>',' ~ ',str(city))
        #city = " ".join(city)
        city = cleanhtml(city)
    
    except:
        city = ""
    
    # address
    try:
        address = re.findall('<p id="address" class="notbold">(.*?)</p>',str(spage))[0]
        address = re.sub('</li>\s*</ul>','',str(address))
        address = re.sub('</li>',' ~ ',str(address))
        #address = " ".join(address)
        address = cleanhtml(address)
    
    except:
        address = ""
    
    # pincode 
    try:
        pincode = re.findall('\d{6}',str(address))[0]
    except:
        pincode = ""
    
    # address substitution
    try:
        address = re.sub('\,\s\d{6}','.',str(address)) # pincode removal
    except:
        address = ""
    
    # contact person
    try:
        contact_person = re.findall('<p id="address">\s*(.*?)\s*</p>',str(spage))[0]
        contact_person = re.sub('</li>\s*</ul>','',str(contact_person))
        #contact_person = re.sub('</li>',' ~ ',str(contact_person))
        #contact_person = " ".join(contact_person)
        contact_person = cleanhtml(contact_person)
        persons = contact_person.split(',')
        contact_persons = list(filter(None,persons))
                
    except:
        contact_person = ""
    
    # phone 
    try:
        phone = re.findall('<i class="fa trin-trin fa-phone" aria-hidden="true"></i>\s*(.*?)\s*</p>',str(spage))[1]
        phone = re.sub('</li>\s*</ul>','',str(phone))
        #phone = re.sub('</li>',' ~ ',str(phone))
        #phone = " ".join(phone)
        phone = cleanhtml(phone)
        mobiles = phone.split(',')
        phones = list(filter(None,mobiles))
        
    except:
        phone = ""
        
    # email
    try:
        email = re.findall('<a  id="address" href="mailto:info@thedetailingmafia.com">(.*?)</a>',str(spage))[0]
        email = re.sub('</li>\s*</ul>','',str(email))
        email = re.sub('</li>',' ~ ',str(email))
        #email = " ".join(email)
        email = cleanhtml(email)
                
    except:
        email = ""
    
    # timing 1
    try:
        timing1 = re.findall('<p id="address">(.*?)</p>',str(spage))[1]
        timing1 = re.sub('</li>\s*</ul>','',str(timing1))
        timing1 = re.sub('</li>',' ~ ',str(timing1))
        #timing1 = " ".join(timing1)
        timing1 = cleanhtml(timing1)
                
    except:
        timing1 = ""
    
    # timing 2
    try:
        timing2 = re.findall('<p id="address">(.*?)</p>',str(spage))[2]
        timing2 = re.sub('</li>\s*</ul>','',str(timing2))
        timing2 = re.sub('</li>',' ~ ',str(timing2))
        #timing2 = " ".join(timing2)
        timing2 = cleanhtml(timing2)
                
    except:
        timing2 = ""
   
    my_dict = {}
    my_dict['Company_Name'] = company_name.strip()
    my_dict['City'] = city.strip()
    my_dict['Address'] = address.strip()
    #my_dict['State'] = state
    my_dict['Pincode'] = pincode
    my_dict['Company_Url'] = main_cat.strip()
    if len(contact_persons) == 1:
        my_dict['Contact_Person_1'] = contact_persons[0].strip()
    else:
        my_dict['Contact_Person_1'] = contact_persons[0].strip()
        my_dict['Contact_Person_2'] = contact_persons[1].strip()
    
    if len(phones) == 1:
        my_dict['Mobile1'] = phones[0].strip()
    elif len(phones) == 2:
        my_dict['Mobile1'] = phones[0].strip()
        my_dict['Mobile2'] = phones[1].strip()
    else:
        my_dict['Mobile1'] = phones[0].strip()
        my_dict['Mobile2'] = phones[1].strip()
        my_dict['Mobile3'] = phones[2].strip()
    my_dict['Email'] = email.strip()
    my_dict['Opening_Closing_Time'] = timing1.strip()
    my_dict['Weekly_Services'] = timing2.strip()
    
    df = pd.DataFrame(my_dict, index=[0])
    list1=list(my_dict.values())
    print(f"writing to csv .... ")
    #count+=1
    with open(out_csv, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header_list,)
        if file.tell() == 0:
            writer.writeheader()# Write the data row
        writer.writerow(my_dict)

### main categories

In [162]:
file_exists = exists('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html')
if file_exists:
        pass
else:
    import requests
    homepage = requests.get(baseurl)
    print(homepage.status_code)
    if homepage.status_code == 200:
            with open('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html','w')as f:
                
                f.write(homepage.text.encode(encoding="utf-8",errors='ignore').decode())
                
HTMLFile = open('/home/justdial/data/input/'+str(input_folder)+'/'+'homepage.html', "r")
hpage = HTMLFile.read()
hpage = re.sub('\\n','',str(hpage))
hpage = re.sub('<!--(.*?)-->','',str(hpage))
hpage = re.sub('#038;','',str(hpage))
m_cut = re.findall('<ul class="portfolio">(.*?)</ul>',str(hpage))
#print(m_cut)
mhash = re.findall('<h2 class="project-info-title".*?>(.*?)</h2>\s*<a class="project-info-button" href="(.*?)"',str(hpage))
main_cat_hash = dict(mhash)
print(main_cat_hash)

{'Agra': 'https://www.thedetailingmafia.com/locate-our-outlets/Agra', 'Ahmedabad': 'https://www.thedetailingmafia.com/locate-our-outlets/Ahmedabad', 'Anantapur': 'https://www.thedetailingmafia.com/locate-our-outlets/Anantapur', 'Anna Nagar Chennai': 'https://www.thedetailingmafia.com/locate-our-outlets/AnnaNagarChennai', 'Attapur': 'https://www.thedetailingmafia.com/locate-our-outlets/Attapur', 'Attibele Bangalore': 'https://www.thedetailingmafia.com/locate-our-outlets/AttibeleBangalore', 'Bannerghatta Bangalore': 'https://www.thedetailingmafia.com/locate-our-outlets/Bannerghatta-Bangalore', 'Bathinda': 'https://www.thedetailingmafia.com/locate-our-outlets/Bhatinda', 'Bhimavaram': 'https://www.thedetailingmafia.com/locate-our-outlets/Bhimavaram', 'Bhopal': 'https://www.thedetailingmafia.com/locate-our-outlets/Bhopal', 'Bhubaneswar': 'https://www.thedetailingmafia.com/locate-our-outlets/Bhuwaneshwar', 'Bommanahalli': 'https://www.thedetailingmafia.com/locate-our-outlets/Bommanahalli', '

In [163]:
for main_cat_url in main_cat_hash:
    main_cat = main_cat_hash[main_cat_url]
    if re.match('http',str(main_cat)):
        pass
    else:
        main_cat = str(baseurl)+str(main_cat)
    print(main_cat)
    main_cat_url = main_cat
    main_cat_url = main_cat_url.replace('/','_').replace(':','_').replace('.','')
    print(main_cat_url)
    
    file_exists = exists('/home/justdial/data/input/'+str(input_folder)+'/'+str(main_cat_url)+'.html')
    if file_exists:
        pass
    else:
        main_cat_page = requests.get(main_cat)
        print(main_cat_page.status_code)
        if main_cat_page.status_code == 200:
            with open('/home/justdial/data/input/'+str(input_folder)+'/'+str(main_cat_url)+'.html','w')as f:
                f.write(main_cat_page.text.encode(encoding="utf-8",errors='ignore').decode())
    HTMLFile = open('/home/justdial/data/input/'+str(input_folder)+'/'+str(main_cat_url)+'.html', "r")
    spage = HTMLFile.read()
    spage = re.sub('\\n','',str(spage))
    fetch_details(spage,main_cat)

https://www.thedetailingmafia.com/locate-our-outlets/Agra
https___wwwthedetailingmafiacom_locate-our-outlets_Agra
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/Ahmedabad
https___wwwthedetailingmafiacom_locate-our-outlets_Ahmedabad
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/Anantapur
https___wwwthedetailingmafiacom_locate-our-outlets_Anantapur
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/AnnaNagarChennai
https___wwwthedetailingmafiacom_locate-our-outlets_AnnaNagarChennai
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/Attapur
https___wwwthedetailingmafiacom_locate-our-outlets_Attapur
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/AttibeleBangalore
https___wwwthedetailingmafiacom_locate-our-outlets_AttibeleBangalore
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/Bannerghatta-Bangalore
https___wwwthedetailingmafiacom_locate

writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/Sarjapur
https___wwwthedetailingmafiacom_locate-our-outlets_Sarjapur
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/Sasaram
https___wwwthedetailingmafiacom_locate-our-outlets_Sasaram
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/Shivalik
https___wwwthedetailingmafiacom_locate-our-outlets_Shivalik
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/Shivamogga
https___wwwthedetailingmafiacom_locate-our-outlets_Shivamogga
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/Sikar-Road-Jaipur
https___wwwthedetailingmafiacom_locate-our-outlets_Sikar-Road-Jaipur
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/Sirsa
https___wwwthedetailingmafiacom_locate-our-outlets_Sirsa
writing to csv .... 
https://www.thedetailingmafia.com/locate-our-outlets/Sri-Muktsar-Sahib
https___wwwthedetailingmafiacom_loca

IndexError: list index out of range